# header

In [1]:
'''
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}

setInterval(ConnectButton,60000);
'''

# from google.colab import drive
# from os import chdir

# drive.mount('/content/drive')
# project_path = '/content/drive/MyDrive/Gproject/MIT_glyco'
# chdir(project_path)

'\nfunction ConnectButton(){\n    console.log("Connect pushed"); \n    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() \n}\n\nsetInterval(ConnectButton,60000);\n'

In [2]:
import pandas as pd
import numpy as np

import os
from os.path import exists
print(os.getcwd()) # current working directory

update = False

D:\Gdrive\내 드라이브\Gproject\MIT_glyco


In [3]:
load_name = "./data/data_for_ml.csv"
protein_list = list(pd.read_csv(load_name).protein.unique())
protein_augmented = [name for name in protein_list if exists(f'./data/data_for_ml(augmented)/{name}.csv')]

dataset = []
for name in protein_augmented:
    load_path = f'./data/data_for_ml(augmented)/{name}.csv'
    dataset.append(pd.read_csv(load_path))
dataset = pd.concat(dataset, axis=0).reset_index(drop=True)

positive = dataset[dataset['positivity']==1]

print("total number of proteins:          ", len(protein_list))
print("total number of augmented proteins:", len(protein_augmented))
print("total number of positive sites:    ", len(positive))

total number of proteins:           272
total number of augmented proteins: 89
total number of positive sites:     185


## parameter settings

In [4]:
from src.Data import *
from src.Preprocessing import *
from src.models import *

variables = xy_variables()
test_size = 0.2
valid_size = test_size/(1-test_size)

import random
random_state = 1
n_cv = 10

hpo_counts = 30
hpo_config = {
    "n_layers" : range(1,11),
    "n_neurons" : [16, 32, 64, 128, 256]
}
metrics = ['epoch', 'time', 'loss', 'val_loss', 'test_loss', 'accuracy', 'precision', 'recall', 'f1']
method = "random"

# model

In [5]:
model_name = 'augmented_cnn_basic'
x_charge = pd.read_csv('./data/from_Krishna/features-all-names.csv').name.to_list()[:-1]

x_cts   = variables.x_cts_original + x_charge
x_cat   = variables.x_cat_original
y_label = variables.y_label

print(f"{len(x_cts)} x_cts")
print(f"{len(x_cat)} x_cat: {x_cat}")
print(f"{len(y_label)} y_label: {y_label}")

data_x, data_y = df_to_dummy(dataset, x_cts, x_cat, y_label)
display(dict(zip(range(len(data_x.columns)), data_x.columns)))

168 x_cts
2 x_cat: ['SEQ', 'SS']
1 y_label: ['positivity']
dummy x shape: (59027, 191)
dummy y shape: (59027, 1)


{0: 'ASA',
 1: 'Phi',
 2: 'Psi',
 3: 'Theta(i-1=>i+1)',
 4: 'Tau(i-2=>i+2)',
 5: 'HSE_alpha_up',
 6: 'HSE_alpha_down',
 7: 'P(C)',
 8: 'P(H)',
 9: 'P(E)',
 10: 'flexibility',
 11: '#',
 12: 'residue_SER_THR',
 13: 'number_of_hydrophobic',
 14: 'number_of_hydrophilic',
 15: 'number_of_polar',
 16: 'number_of_aromatic',
 17: 'number_of_aliphatic',
 18: 'number_of_charged',
 19: 'number_of_positive',
 20: 'number_of_negative',
 21: 'number_of_g',
 22: 'number_of_v',
 23: 'number_of_s',
 24: 'number_of_n',
 25: 'number_of_l',
 26: 'number_of_p',
 27: 'number_of_A',
 28: 'number_of_b',
 29: 'number_of_d',
 30: 'number_of_e',
 31: 'number_of_f',
 32: 'number_of_ala',
 33: 'number_of_cys',
 34: 'number_of_asp',
 35: 'number_of_glu',
 36: 'number_of_phe',
 37: 'number_of_his',
 38: 'number_of_ile',
 39: 'number_of_lys',
 40: 'number_of_leu',
 41: 'number_of_met',
 42: 'number_of_asn',
 43: 'number_of_gln',
 44: 'number_of_arg',
 45: 'number_of_ser',
 46: 'number_of_thr',
 47: 'number_of_val',


In [6]:
window_size = 10

seq_input  = []
seq_output = []

for name in protein_augmented:
    load_path = f'./data/data_for_ml(augmented)/{name}.csv'
    temp = pd.read_csv(load_path)
    temp_x, temp_y = custom_dummy(temp, x_cts, x_cat, y_label)
    
    temp_input, temp_output = data_to_sequence(temp_x, temp_y, window_size)
    seq_input.append(temp_input)
    seq_output.append(temp_output)
    
seq_input  = np.concatenate(seq_input, axis=0)
seq_output = np.concatenate(seq_output, axis=0)

print(f'rnn input shape : {seq_input.shape}')
print(f'rnn output shape: {seq_output.shape}')

rnn input shape : (8910, 21, 191)
rnn output shape: (8910, 1)


## cross validation

In [8]:
# select n_layers and n_neurons as the best values of HPO
cv_path = f'./result/cv_result_{model_name}_{n_cv}.csv'

cv_result = pd.DataFrame([], columns=metrics)
if not exists(cv_path) or update:
    for i in range(n_cv):
        print(f"\n{i+1}th iteration")
        random.seed(i+1)
        train_x, train_y, test_x, test_y, _, _ = stratified_split(seq_input, seq_output, 
                                                              test_size=test_size, random_state=i+1, 
                                                              scale_x=x_cts, scale_y=[])
        train_x, train_y = up_sampling(train_x, train_y)

        history_size = train_x.shape[1]
        x_dim = train_x.shape[2]
        y_dim = train_y.shape[1]
        save_path  = f'./h5/{model_name}_cv_{i+1}of{n_cv}.h5'
        
        model = CNN1D(history_size, x_dim, y_dim)
        model.build()
        if not exists(save_path) or update:
            model.train(train_x, train_y, valid_size, save_path=save_path)
        else:
            model.load_model(save_path)

        epoch = model.epoch
        time = model.time
        loss = model.loss
        val_loss = model.val_loss
        test_loss, accuracy, precision, recall, f1 = model.evaluate(test_x, test_y)
        cv = pd.DataFrame([[epoch, time, loss, val_loss, test_loss, accuracy, 
                             precision[1], recall[1], f1[1]]], columns=metrics)
        cv_result = pd.concat([cv_result, cv], axis=0)
    cv_result = cv_result.reset_index(drop=True)
    cv_result.to_csv(cv_path, index=False)    
    
else:
    cv_result = pd.read_csv(cv_path)

display(cv_result)


1th iteration
train/test dataset: <class 'numpy.ndarray'>

train: (7128, 21, 191) (7128, 1)
check scale: 0.0 1.0

test: (1782, 21, 191) (1782, 1)
check scale: 0.0 1.1
up-sampled train dataset: (13960, 21, 191) (13960, 1)
model has been restored from ./h5/augmented_cnn_basic_cv_1of10.h5

2th iteration
train/test dataset: <class 'numpy.ndarray'>

train: (7128, 21, 191) (7128, 1)
check scale: 0.0 1.0

test: (1782, 21, 191) (1782, 1)
check scale: -0.08089430894308944 1.1
up-sampled train dataset: (13960, 21, 191) (13960, 1)
model has been saved to ./h5/augmented_cnn_basic_cv_2of10.h5

3th iteration
train/test dataset: <class 'numpy.ndarray'>

train: (7128, 21, 191) (7128, 1)
check scale: 0.0 1.0

test: (1782, 21, 191) (1782, 1)
check scale: 0.0 1.2375476320815681
up-sampled train dataset: (13960, 21, 191) (13960, 1)
model has been saved to ./h5/augmented_cnn_basic_cv_3of10.h5

4th iteration
train/test dataset: <class 'numpy.ndarray'>

train: (7128, 21, 191) (7128, 1)
check scale: 0.0 1.0


,epoch,time,loss,val_loss,test_loss,accuracy,precision,recall,f1
0,21,1.968,0.000403,0.020245,0.530656,97.42,23.53,10.81,14.81
1,9,1.539,0.009857,0.028226,0.400710,96.91,17.86,13.51,15.38
2,5,1.308,0.008609,0.035927,0.385016,97.03,16.67,10.81,13.11
3,22,1.891,0.017765,0.011632,0.342904,97.14,25.00,18.92,21.54
4,21,1.903,0.009809,0.017653,0.391630,97.25,28.57,21.62,24.62
5,27,2.072,0.007991,0.020849,0.433319,97.31,26.09,16.22,20.00
6,16,1.719,0.023107,0.032719,0.311069,96.91,17.86,13.51,15.38
7,22,1.992,0.017414,0.024198,0.428208,97.59,25.00,8.11,12.24
8,40,2.714,0.000926,0.015975,0.476106,97.87,47.06,21.62,29.63
9,50,3.255,0.001629,0.018109,0.415293,97.08,17.39,10.81,13.33


In [ ]:
from src.graph_plot import *

for i in range(n_cv):
    print(f"\n{i+1}th iteration")
    random.seed(i+1)
    train_x, train_y, test_x, test_y, _, _ = stratified_split(rnn_input, rnn_output, 
                                                          test_size=test_size, random_state=i+1, 
                                                          scale_x=x_cts, scale_y=[])
    train_x, train_y = up_sampling(train_x, train_y)

    history_size = train_x.shape[1]
    x_dim = train_x.shape[2]
    y_dim = train_y.shape[1]
    save_path  = f'./h5/{model_name}_{rnn_layers}_{rnn_neurons}_{dnn_layers}_{dnn_neurons}_cv_{i+1}of{n_cv}.h5'

    model = RNN(history_size, x_dim, y_dim)
    model.build(rnn_layers, rnn_neurons, dnn_layers, dnn_neurons)
    model.load_model(save_path)
    
    prediction = model.model.predict(test_x, verbose=0)
    y_pred = prediction.round(0).astype(int)
    y_real = test_y
    
    plot_confusion(y_real, y_pred, title=model_name+f"_{i+1}", label=["Positive","Negative"])